# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In this GitHub repository, there is a file called `allergies.json` that contains a list of patient allergies.  You will need to download this [file from here](https://raw.githubusercontent.com/paulboal/hds5210-2023/main/week06/allergies.json) and then upload it into Google Colab to run these examples. It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [311]:
import json
ALLERGIES_FILE="allergies.json"

In [312]:
def allergy_count(json_file):
  # Open the allergies JSON file for reading
  with open(json_file, 'r') as file:
    # Load the JSON data into an object
    allergies_data = json.load(file)

    # Check if the "entry" key exists and it's a list
    if 'entry' in allergies_data and isinstance(allergies_data['entry'], list):
        # Count the number of items in the "entry" list
        number_of_entries = len(allergies_data['entry'])
        return number_of_entries
    else:
        # Return 0 if "entry" key is missing or not a list
        return 0

In [313]:
allergy_count(ALLERGIES_FILE)

4

In [314]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [315]:
def patient_count(json_file):
  # Open the allergies JSON file for reading
  with open(json_file, 'r') as file:
      # Load the JSON data into a Python object
      allergies_data = json.load(file)

      # Check if the "entry" key exists and and also it's a list
      if 'entry' in allergies_data and isinstance(allergies_data['entry'], list):
          # Create a set to store unique patient names
          unique_patients = set()

          # Loop through the entries and collect unique patient names
          for entry in allergies_data['entry']:
              if 'resource' in entry and 'patient' in entry['resource']:
                  patient_name = entry['resource']['patient']['display']
                  unique_patients.add(patient_name)

          # Count the number of unique patients
          num_unique_patients = len(unique_patients)
          return num_unique_patients
      else:
          # If "entry" key is missing or if it is not a list return 0
          return 0


In [316]:
patient_count(ALLERGIES_FILE)

2

In [317]:
assert type(patient_count(ALLERGIES_FILE)) == int
assert patient_count(ALLERGIES_FILE) == 2

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [318]:
def allergy_per_patient(json_file):
  # Open the allergies JSON file for reading
  with open(json_file, 'r') as file:
      # Load the JSON data into an object
      allergies_data = json.load(file)

      # Check if the "entry" key exists and it's a list
      if 'entry' in allergies_data and isinstance(allergies_data['entry'], list):
          # Create an empty dictionary to store allergies per patient
          allergies_by_patient = {}

          # Loop through the entries and count allergies for each patient
          for entry in allergies_data['entry']:
              if 'resource' in entry and 'patient' in entry['resource']:
                  patient_name = entry['resource']['patient']['display']

                  # Check if the patient is already present in the dictionary
                  if patient_name in allergies_by_patient:
                      allergies_by_patient[patient_name] += 1
                  else:
                      allergies_by_patient[patient_name] = 1

          return allergies_by_patient
      else:
          # If "entry" key is missing or if it is not a list return empty object
          return {}


In [319]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

In [320]:
assert type(allergy_per_patient(ALLERGIES_FILE)) == dict
assert allergy_per_patient(ALLERGIES_FILE) == {'Paul Boal': 1, 'Jason Argonaut': 3}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [321]:
def allergy_list(json_file):
  # Open the JSON file for reading
  with open(json_file, 'r') as file:
      # Load the JSON data into a Python object
      allergies_data = json.load(file)

      # Check to see if the "entry" key exists and if it is a list.
      if 'entry' in allergies_data and isinstance(allergies_data['entry'], list):
          # Create an empty list to store the output
          output_list = []

          # Iterate through the entries and extract patient name, allergy, and reaction
          for entry in allergies_data['entry']:
              if 'resource' in entry and 'patient' in entry['resource']:
                  patient_name = entry['resource']['patient']['display']
                  allergy = entry['resource']['substance']['text']

                  # Initialize reaction and manifestation to None
                  reaction = None
                  manifestation = None

                  # Check if there is at least one reaction
                  if 'reaction' in entry['resource'] and isinstance(entry['resource']['reaction'], list) and len(entry['resource']['reaction']) > 0:
                      reaction = entry['resource']['reaction'][0]

                      # Check if there is at least one manifestation in the reaction
                      if 'manifestation' in reaction and isinstance(reaction['manifestation'], list):
                          manifestation = reaction['manifestation'][0]['text']

                  # Create a list with patient name, allergy, and reaction (or None)
                  entry_info = [patient_name, allergy, manifestation]
                  output_list.append(entry_info)

          return output_list
      else:
          # "entry" key is missing or not a list
          return []


In [322]:
allergy_list(ALLERGIES_FILE)

[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]

In [323]:
assert allergy_list(ALLERGIES_FILE) == [['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]


### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  You can solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [324]:
def allergy_reaction(json_file, patient, substance):
    # Get the list of allergies by using the allergy_list function
    list_of_allergies = allergy_list(json_file)

    # loop through the allergies to find the matching patient and substance
    for entry_info in list_of_allergies:
        entry_patient, entry_substance, reaction = entry_info

        # Check if the patient and substance match the specified values
        if entry_patient == patient and entry_substance == substance:
            return reaction  # Return the reaction if found

    # If a matching entry is not found, return None
    return None


In [325]:
allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G')

'Hives'

In [326]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to save this notebook file back to GitHub.  To do that in Google Colab:
1. File -> Save a Copy in GitHub
2. Make sure your HDS5210 repository is selected
3. Make sure the file name includes the week number like this: `week06/week06_assignment_2.ipynb`
4. Add a commit message that means something

**Be sure week names are lowercase and use a two digit week number!!**

**Be sure you use the same file name provided by the instructor!!**

